In [46]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
#Read in csv as dataframe
df = pd.read_csv('/Users/ethanrowe/Dev/springboard_data/KSU Assessment Page Views Fall 2018.csv', nrows = 10000)

In [48]:
#Eliminate Outliers using the 1.5*interquartile rule
IQR = float(df['duration'].quantile([0.75])) - float(df['duration'].quantile([0.25]))
upper_bound = float(df['duration'].quantile([0.75])) + 1.5*IQR
lower_bound = float(df['duration'].quantile([0.25])) - 1.5*IQR

df_clean = df[(df['duration'] < upper_bound) & (df['duration'] > lower_bound)]

In [19]:
print(df_clean.shape)
print("Null Values: " + str(df_clean.isnull().values.any()))
#df_clean.to_csv('KSU Assessment Page Views Fall 2018 Clean.csv')

(8846, 13)
Null Values: False


In [22]:
#Group the csv file to find the average page view times for each page and each student
# 'assessment_attempt_id' not included
page_df = pd.DataFrame(df_clean.groupby(['canvas_assignment_id', 'page_id'])['duration'].mean())
print('\nGrouped by Page')
page_df.columns = ['Average Duration']
print(page_df.shape)

user_df = pd.DataFrame(df_clean.groupby(['user_param_external_user_id', 'canvas_assignment_id', 'page_id'])['duration'].mean())
print('\nGrouped by User')
user_df.columns = ['Average Duration']
print(user_df.shape)


#page_df.to_csv('KSU Assessment Page Duration Averages.csv')
#user_df.to_csv('KSU Assessment Page Views by User.csv')





Grouped by Page
(7058, 1)

Grouped by User
(8349, 1)


In [20]:
total = 0
rows_to_drop = []
for index1, row1 in page_df.iterrows():
    for index2, row2 in user_df.iterrows():
        total += 1
        """
        li = list(index2)
        li.pop(0) #removes the user_id so it can compare [1,1,1] to list((1,1,1)) to find a page and attempt match
        if li == list(index1):
            if row2['Average Duration'] < row1['Average Duration']:
                #Remove this row
                rows_to_drop.append(index2)
        """        
                
user_df = user_df.drop(rows_to_drop, axis = 0)
print(user_df)

KeyboardInterrupt: 

In [45]:
#Determine if there are missing values missing values
total =  []
for chunk in pd.read_csv('KSU Assessment Page Views Fall 2018 Clean.csv', chunksize = 1000):
    if chunk.isnull().values.any():
        total.append(chunk)
        
#There are missing values in the whole dataframe
#Need to check user_id, canvas_assignment_d, canvas_attempt_id, page_ids
#There are no missing values in the columns that we are taking an interest in for any of our calculations
#Checked each column for missing values and each column had none

FileNotFoundError: [Errno 2] File b'KSU Assessment Page Views Fall 2018 Clean.csv' does not exist: b'KSU Assessment Page Views Fall 2018 Clean.csv'